# Mokai Microseismicity Event Map, for Mercury Energy

## No report is required for this project, but this will help visualise data when comments are requested by Mercury

In [ ]:
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.img_tiles import OSM

import pandas as pd
import numpy as np
import pyproj
from math import floor

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
from matplotlib import patheffects
import matplotlib.gridspec as gridspec
import warnings; warnings.simplefilter('ignore')

%matplotlib inline

In [ ]:
#full width notebook display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

**Plot distance scale (and north arrow) on plot. Reference in scale_bar function documentation**

In [ ]:
def utm_from_lon(lon):
    """
    utm_from_lon - UTM zone for a longitude

    Not right for some polar regions (Norway, Svalbard, Antartica)

    :param float lon: longitude
    :return: UTM zone number
    :rtype: int
    """
    return floor( ( lon + 180 ) / 6) + 1

def scale_bar(ax, proj, length, location=(0.5, 0.05), linewidth=3,
              units='km', m_per_unit=1000):
    """

    http://stackoverflow.com/a/35705477/1072212
    ax is the axes to draw the scalebar on.
    proj is the projection the axes are in
    location is center of the scalebar in axis coordinates ie. 0.5 is the middle of the plot
    length is the length of the scalebar in km.
    linewidth is the thickness of the scalebar.
    units is the name of the unit
    m_per_unit is the number of meters in a unit
    """
    # find lat/lon center to find best UTM zone
    x0, x1, y0, y1 = ax.get_extent(proj.as_geodetic())
    # Projection in metres
    utm = ccrs.UTM(utm_from_lon((x0+x1)/2))
    # Get the extent of the plotted area in coordinates in metres
    x0, x1, y0, y1 = ax.get_extent(utm)
    # Turn the specified scalebar location into coordinates in metres
    sbcx, sbcy = x0 + (x1 - x0) * location[0], y0 + (y1 - y0) * location[1]
    # Generate the x coordinate for the ends of the scalebar
    bar_xs = [sbcx - length * m_per_unit/2, sbcx + length * m_per_unit/2]
    # buffer for scalebar
    buffer = [patheffects.withStroke(linewidth=5, foreground="w")]
    # Plot the scalebar with buffer
    ax.plot(bar_xs, [sbcy, sbcy], transform=utm, color='k',
        linewidth=linewidth, path_effects=buffer)
    # buffer for text
    buffer = [patheffects.withStroke(linewidth=3, foreground="w")]
    # Plot the scalebar label
    t0 = ax.text(sbcx, sbcy, str(length) + ' ' + units, transform=utm,
        horizontalalignment='center', verticalalignment='bottom',
        path_effects=buffer, zorder=2)
    left = x0+(x1-x0)*0.05
    # Plot the N arrow
    t1 = ax.text(left, sbcy, u'\u25B2\nN', transform=utm,
        horizontalalignment='center', verticalalignment='bottom',
        path_effects=buffer, zorder=2)
    # Plot the scalebar without buffer, in case covered by text buffer
    ax.plot(bar_xs, [sbcy, sbcy], transform=utm, color='k',
        linewidth=linewidth, zorder=3)
    
# or to use m instead of km
# scale_bar(ax, ccrs.Mercator(), 100000, m_per_unit=1, units='m')
# or to use miles instead of km
# scale_bar(ax, ccrs.Mercator(), 60, m_per_unit=1609.34, units='miles')

#need specific font to show triangle for north arrow
mpl.rc('font', family='DejaVu Sans')

**Plot area**

In [ ]:
lonmin = 175.881907
lonmax = 175.968088
latmin = -38.544951
latmax = -38.486414

## Seismicity from Local Network

**This views a csv data file from a recent download**

In [ ]:
datafile = 'revised_download_24.csv'

In [ ]:
hyp = pd.read_csv(datafile, parse_dates={"ot" : ['year', 'month', 'day', 'hour', 'minute', 'second']}, index_col='ot')

In [ ]:
hyp.index = pd.to_datetime(hyp.index, format='%Y %m %d %H %M %S.%f')

In [ ]:
hyp.head()

In [ ]:
hyp['magnitude'].max()

In [ ]:
hyp['magnitude'].count()

**Plot hypocentres on Open Street Map basemap**

In [ ]:
fig = plt.figure(dpi=100)
#imagery
imagery = OSM()
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 14)
#ax.gridlines(draw_labels=True)

#field boundary
names = ['lon', 'lat']
field = pd.read_csv('field.dat', names=names)
plt.plot(field['lon'], field['lat'], color='black', marker=None, linewidth=1, transform=ccrs.Geodetic())

#hypocentres, symbol size=magnitude squared, as scatter symbol size is symbol area, square gives 'pleasing' image
plt.scatter(hyp['longitude'], hyp['latitude'], color='red', marker='o', s=hyp['magnitude']**2, transform=ccrs.Geodetic())

#seismographs
names = ['lon', 'lat', 'code']
seis = pd.read_csv('sites.dat', sep='\s+', names=names)
plt.plot(seis['lon'], seis['lat'], color='gray', markeredgewidth=0.0, linestyle='None', marker='^', markersize=5, transform=ccrs.Geodetic())

#scale bar + north arrow
#scale_bar(ax, ccrs.Mercator(), 1)

plt.savefig('map.png', dpi=200)

**Magnitude-time**

In [ ]:
#magnitude vs time
ax = hyp.plot(y='magnitude', color='black', marker='o', linewidth=0.3, linestyle='None', legend=False, figsize=(15,5))
ax.xaxis.grid()
ax.yaxis.grid()

ax.set_ylim([0,3])
ax.set_ylabel('Magnitude')
ax.tick_params(labelright = True)

plt.tight_layout()
#plt.savefig('time-mag.png', dpi=200)